In [1]:
# Tế bào 1: Import thư viện
import pandas as pd
import numpy as np
import re
import os
from tqdm.notebook import tqdm
import warnings

# Cấu hình để hiển thị tất cả các cột và định dạng float
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
# Bỏ qua các cảnh báo không quan trọng
warnings.filterwarnings('ignore')

print("Libraries imported and configured successfully.")


Libraries imported and configured successfully.


In [2]:
# Tế bào 2: Tải dữ liệu từ hai nguồn (SR Legacy và Foundation)

# --- Objective ---
# Tải tất cả các file CSV cần thiết từ cả hai bộ dữ liệu USDA,
# chỉ lấy các cột cần thiết để tối ưu hóa việc sử dụng bộ nhớ RAM 8GB.

# --- Implementation ---
print("--- Step 1: Extracting data from USDA sources ---")
# !!! QUAN TRỌNG: Hãy đảm bảo tên thư mục bên dưới là chính xác !!!
BASE_DATA_PATH = os.path.expanduser('~/Desktop/')
SR_LEGACY_PATH = os.path.join(BASE_DATA_PATH, 'FoodData_Central_sr_legacy_food_csv_2018-04') # Thay bằng tên thư mục của em
FOUNDATION_PATH = os.path.join(BASE_DATA_PATH, 'FoodData_Central_foundation_food_csv_2025-04-24') # Thay bằng tên thư mục của em
FOOD_DOT_COM_PATH = os.path.join(BASE_DATA_PATH, 'Food.com Recipes and Interactions')

--- Step 1: Extracting data from USDA sources ---


In [3]:
try:
    # Tải từ SR Legacy
    sr_food_df = pd.read_csv(os.path.join(SR_LEGACY_PATH, 'food.csv'), usecols=['fdc_id', 'description'])
    sr_food_nutrient_df = pd.read_csv(os.path.join(SR_LEGACY_PATH, 'food_nutrient.csv'), usecols=['fdc_id', 'nutrient_id', 'amount'])
    
    # Tải từ Foundation
    foundation_food_df = pd.read_csv(os.path.join(FOUNDATION_PATH, 'food.csv'), usecols=['fdc_id', 'description'])
    foundation_food_nutrient_df = pd.read_csv(os.path.join(FOUNDATION_PATH, 'food_nutrient.csv'), usecols=['fdc_id', 'nutrient_id', 'amount'])
    
    # Tải file nutrient chung (chúng giống nhau)
    nutrient_df = pd.read_csv(os.path.join(SR_LEGACY_PATH, 'nutrient.csv'), usecols=['id', 'name'])
    
    print("All USDA source files loaded successfully.")
except Exception as e:
    print(f"ERROR loading files: {e}")
    print("Please double-check your folder names and paths.")

All USDA source files loaded successfully.


In [4]:

# --- Verification 1 ---
print("\n--- Verification 1 ---")
print("Initial shapes of loaded dataframes:")
print(f"SR Legacy Foods: food={sr_food_df.shape}, food_nutrient={sr_food_nutrient_df.shape}")
print(f"Foundation Foods: food={foundation_food_df.shape}, food_nutrient={foundation_food_nutrient_df.shape}")
print(f"Nutrient table: {nutrient_df.shape}")


--- Verification 1 ---
Initial shapes of loaded dataframes:
SR Legacy Foods: food=(7793, 2), food_nutrient=(644125, 3)
Foundation Foods: food=(74175, 2), food_nutrient=(155243, 3)
Nutrient table: (474, 2)


In [5]:
# --- Implementation & Verification ---

# === 2.1: Hợp nhất Dữ liệu Thô từ USDA ===
print("--- 2.1: Combining SR Legacy and Foundation data ---")
combined_food_df = pd.concat([sr_food_df, foundation_food_df], ignore_index=True).drop_duplicates(subset=['fdc_id'])
combined_food_nutrient_df = pd.concat([sr_food_nutrient_df, foundation_food_nutrient_df], ignore_index=True)
print(f"Combined food data shape: {combined_food_df.shape}")
print(f"Combined nutrient data shape: {combined_food_nutrient_df.shape}")
print("-" * 80)

--- 2.1: Combining SR Legacy and Foundation data ---
Combined food data shape: (81968, 2)
Combined nutrient data shape: (799368, 3)
--------------------------------------------------------------------------------


In [6]:

# === 2.2: Lọc, Pivot và Hợp nhất để tạo Bảng Thô ===
print("\n--- 2.2: Filtering, Pivoting, and Merging to create Raw Lookup Table ---")
NUTRIENT_IDS = {
    'calories': 1008, 'protein_g': 1003, 'fat_g': 1004,
    'carbs_g': 1005, 'fiber_g': 1079, 'sugar_g': 2000
}
food_nutrient_filtered_df = combined_food_nutrient_df[combined_food_nutrient_df['nutrient_id'].isin(NUTRIENT_IDS.values())].copy()
nutrient_id_to_col_name = {v: k for k, v in NUTRIENT_IDS.items()}
nutrient_pivot_df = food_nutrient_filtered_df.pivot_table(index='fdc_id', columns='nutrient_id', values='amount', aggfunc='first')
nutrient_flat_df = nutrient_pivot_df.reset_index().rename(columns=nutrient_id_to_col_name)
nutrient_flat_df.columns.name = None
usda_lookup_raw_df = pd.merge(combined_food_df.dropna(subset=['description']), nutrient_flat_df, on='fdc_id', how='inner')
print(f"Raw lookup table created. Shape: {usda_lookup_raw_df.shape}")
print("-" * 80)


--- 2.2: Filtering, Pivoting, and Merging to create Raw Lookup Table ---
Raw lookup table created. Shape: (13634, 8)
--------------------------------------------------------------------------------


In [7]:
# Verification
print(f"Shape of food_nutrient table after filtering: {food_nutrient_filtered_df.shape}")
print("Value counts per nutrient:")
print(food_nutrient_filtered_df['nutrient_id'].value_counts())
print("-" * 60)

Shape of food_nutrient table after filtering: (53081, 3)
Value counts per nutrient:
nutrient_id
1004    12352
1079     9435
1003     9169
1005     8145
1008     7928
2000     6052
Name: count, dtype: int64
------------------------------------------------------------


In [143]:
# Verification
print("Pivot and flatten complete.")
print(f"Shape of flattened table: {nutrient_flat_df.shape}")
print("First 5 rows of flattened table (contains NaNs):")
display(nutrient_flat_df.head())
print("-" * 60)

Pivot and flatten complete.
Shape of flattened table: (13634, 7)
First 5 rows of flattened table (contains NaNs):


,fdc_id,protein_g,fat_g,carbs_g,calories,fiber_g,sugar_g
0,167512,5.88,13.24,41.18,307.00,1.20,5.88
1,167513,4.34,11.27,53.42,330.00,1.40,21.34
2,167514,6.10,3.70,79.80,377.00,NaN,NaN
3,167515,8.00,1.80,46.00,232.00,NaN,NaN
4,167516,6.58,9.22,41.05,273.00,2.20,4.30


------------------------------------------------------------


In [8]:
# Verification
print(f"Shape of raw lookup table after merge: {usda_lookup_raw_df.shape}")
print("First 5 rows of raw lookup table:")
display(usda_lookup_raw_df.head())
print("-" * 60)

Shape of raw lookup table after merge: (13634, 8)
First 5 rows of raw lookup table:


,fdc_id,description,protein_g,fat_g,carbs_g,calories,fiber_g,sugar_g
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",5.88,13.24,41.18,307.00,1.20,5.88
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger...",4.34,11.27,53.42,330.00,1.40,21.34
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa...",6.10,3.70,79.80,377.00,NaN,NaN
3,167515,"George Weston Bakeries, Thomas English Muffins",8.00,1.80,46.00,232.00,NaN,NaN
4,167516,"Waffles, buttermilk, frozen, ready-to-heat",6.58,9.22,41.05,273.00,2.20,4.30


------------------------------------------------------------


In [82]:
# === Step 3.4: Tổng hợp Dữ liệu Thông minh ===
print("\n--- Step 3.4: Performing Intelligent Aggregation ---")
# 1. Xác định các cột số cần tổng hợp
numeric_cols = list(NUTRIENT_IDS.keys())
# 2. Thay thế các giá trị 0 bằng NaN để không làm ảnh hưởng đến phép tính trung bình
usda_lookup_raw_df[numeric_cols] = usda_lookup_raw_df[numeric_cols].replace(0, np.nan)
# 3. Tạo cột chữ thường để nhóm
usda_lookup_raw_df['description_lower'] = usda_lookup_raw_df['description'].str.lower()
# 4. Thực hiện groupby và tính trung bình (hàm mean() sẽ bỏ qua NaN)
usda_aggregated_lookup_df = usda_lookup_raw_df.groupby('description_lower', as_index=False)[numeric_cols].mean()
# 5. Điền 0 cho các giá trị NaN còn lại sau khi tính trung bình
usda_aggregated_lookup_df.fillna(0, inplace=True)


--- Step 3.4: Performing Intelligent Aggregation ---


In [83]:

# --- FINAL VALIDATION ---
print("\nFinal Aggregated USDA Nutrient Lookup Dictionary created successfully.")
print("\n--- FINAL VALIDATION ---")
print(f"Shape of the final aggregated lookup table: {usda_aggregated_lookup_df.shape}")

print("\nVerifying 'hummus' nutrient profile:")
display(usda_aggregated_lookup_df[usda_aggregated_lookup_df['description_lower'].str.contains('hummus', na=False)].head())

print("\nVerifying 'chicken breast' nutrient profile:")
display(usda_aggregated_lookup_df[usda_aggregated_lookup_df['description_lower'].str.contains('chicken', na=False)].head())

print("\nFinal check for null values:")
if usda_aggregated_lookup_df.isnull().sum().sum() == 0:
    print("No null values found.")
else:
    print("WARNING: Null values still exist.")

print("\nDescriptive statistics of the final lookup table:")
display(usda_aggregated_lookup_df.describe())


Final Aggregated USDA Nutrient Lookup Dictionary created successfully.

--- FINAL VALIDATION ---
Shape of the final aggregated lookup table: (10090, 7)

Verifying 'hummus' nutrient profile:


,description_lower,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
5248,hummus,0.00,0.00,17.14,0.00,0.00,0.00
5249,hummus - nfy1213zx,0.00,0.00,0.00,0.00,5.50,0.00
5250,hummus - nfy12140q,0.00,0.00,0.00,0.00,5.70,0.00
5251,hummus - nfy12141f,0.00,0.00,0.00,0.00,5.40,0.00
5252,hummus - nfy12141z,0.00,0.00,0.00,0.00,5.50,0.00



Verifying 'chicken breast' nutrient profile:


,description_lower,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
114,"applebee's, chicken tenders platter",297.00,19.62,16.24,17.98,1.00,0.36
115,"applebee's, chicken tenders, from kids' menu",296.00,19.25,16.15,18.36,1.20,0.39
298,"babyfood, dinner, apples and chicken, strained",65.00,2.16,1.38,10.88,1.80,6.06
304,"babyfood, dinner, broccoli and chicken, junior",62.00,3.59,2.48,6.34,1.40,1.43
305,"babyfood, dinner, chicken and rice",51.00,1.60,0.90,9.20,1.10,0.55



Final check for null values:
No null values found.

Descriptive statistics of the final lookup table:


,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
count,10090.00,10090.00,10090.00,10090.00,10090.00,10090.00
mean,171.09,11.74,9.63,15.63,1.91,4.54
std,174.69,11.09,15.67,24.31,3.89,12.00
min,0.00,0.00,0.00,-0.70,0.00,0.00
25%,20.00,1.20,0.75,0.00,0.00,0.00
50%,129.00,8.62,3.59,2.65,0.10,0.00
75%,270.00,21.94,12.13,20.20,2.70,2.50
max,902.00,88.32,100.00,100.00,79.00,99.80


In [9]:

# === Step 4.4: Tổng hợp Dữ liệu Thông minh theo Gốc Ngữ nghĩa ===
print("\n--- Step 4.4: Performing Intelligent Semantic Aggregation ---")

# --- 4.4.1: Xử lý Dữ liệu Lỗi và Nhiễu ---
print("\n--- 4.4.1: Cleaning Noisy Data ---")
numeric_cols = list(NUTRIENT_IDS.keys())
# Loại bỏ các hàng có giá trị dinh dưỡng âm (phi vật lý)
initial_rows = len(usda_lookup_raw_df)
for col in numeric_cols:
    usda_lookup_raw_df = usda_lookup_raw_df[usda_lookup_raw_df[col] >= 0]
cleaned_rows = len(usda_lookup_raw_df)
print(f"Removed {initial_rows - cleaned_rows} rows with negative nutritional values.")


--- Step 4.4: Performing Intelligent Semantic Aggregation ---

--- 4.4.1: Cleaning Noisy Data ---
Removed 7700 rows with negative nutritional values.


In [10]:
# Verification (Before)
print("Min values before cleaning negative values:")
print(usda_lookup_raw_df[numeric_cols].min())

initial_rows = len(usda_lookup_raw_df)
for col in numeric_cols:
    usda_lookup_raw_df = usda_lookup_raw_df[usda_lookup_raw_df[col] >= 0]
print(f"Removed {initial_rows - len(usda_lookup_raw_df)} rows with negative nutritional values.")
# Verification (After)
print("\nMin values after cleaning:")
print(usda_lookup_raw_df[numeric_cols].min())
print("-" * 60)

Min values before cleaning negative values:
calories    0.00
protein_g   0.00
fat_g       0.00
carbs_g     0.00
fiber_g     0.00
sugar_g     0.00
dtype: float64
Removed 0 rows with negative nutritional values.

Min values after cleaning:
calories    0.00
protein_g   0.00
fat_g       0.00
carbs_g     0.00
fiber_g     0.00
sugar_g     0.00
dtype: float64
------------------------------------------------------------


In [12]:
# === Step 4.4: Trích xuất Gốc Ngữ nghĩa Thông minh (Intelligent Root Extraction) ===
print("\n--- Step 4.4: Implementing Intelligent Semantic Root Extraction ---")

def extract_semantic_root_v2(description: str) -> str:
    if not isinstance(description, str):
        return ""
        
    # Chuyển về chữ thường và tách các phần bởi dấu phẩy
    parts = [part.strip() for part in description.lower().split(',')]
    
    # TH1: Mô tả ngắn, không có dấu phẩy (ví dụ: 'Hummus')
    if len(parts) == 1:
        # Loại bỏ các phần trong ngoặc đơn
        return re.sub(r'\(.*\)', '', parts[0]).strip()
        
    # TH2: Mô tả có cấu trúc (ví dụ: 'Nuts, almond butter, plain...')
    # Giả định phần thứ 2 là tên chính, phần đầu là danh mục.
    # Chúng ta sẽ ưu tiên lấy phần thứ 2.
    main_name = parts[1]
    
    # Kiểm tra xem phần đầu (danh mục) có phải là một từ chung chung không.
    # Nếu không, nó có thể là một phần quan trọng của tên (ví dụ: 'Beef, ground')
    generic_categories = ['nuts', 'beverages', 'spices and herbs', 'fats and oils', 'sweets']
    if parts[0] not in generic_categories and not parts[0].startswith('babyfood'):
        # Nếu danh mục không chung chung, hãy ghép nó với tên chính
        # Ví dụ: 'Beef, ground' -> 'beef ground'
        main_name = f"{parts[0]} {parts[1]}"
        
    # Loại bỏ các phần mô tả không cần thiết phía sau (ví dụ: 'with salt added')
    main_name = main_name.split(' with ')[0]
    main_name = main_name.split(' without ')[0]
    
    # Loại bỏ các phần trong ngoặc đơn
    main_name = re.sub(r'\(.*\)', '', main_name).strip()
    
    return main_name

# Áp dụng hàm mới
usda_lookup_raw_df['semantic_root'] = usda_lookup_raw_df['description'].apply(extract_semantic_root_v2)


--- Step 4.4: Implementing Intelligent Semantic Root Extraction ---


In [13]:
# Verification 2.3
print("Verifying the new root extraction logic:")
display(usda_lookup_raw_df[usda_lookup_raw_df['description'].str.contains('Nuts, almond butter|Beef, ground, 85%|Hummus, commercial', case=False, regex=True)][['description', 'semantic_root']].head())
print("-" * 80)

Verifying the new root extraction logic:


,description,semantic_root
1076,"Nuts, almond butter, plain, without salt added",almond butter
1091,"Nuts, almond butter, plain, with salt added",almond butter
4284,"Beef, ground, 85% lean meat / 15% fat, raw (In...",beef ground
6520,"Beef, ground, 85% lean meat / 15% fat, patty, ...",beef ground
6521,"Beef, ground, 85% lean meat / 15% fat, patty, ...",beef ground


--------------------------------------------------------------------------------


In [14]:
# --- Verification 4.4 ---
print("\n--- Verification 4.4: V3 Root Extraction ---")
print("Verifying the new logic on 'butter', 'beef', and 'hummus':")
display(usda_lookup_raw_df[usda_lookup_raw_df['description'].str.contains('butter|beef, ground|hummus', case=False, regex=True)][['description', 'semantic_root']].sample(15, random_state=1))
print("-" * 80)


--- Verification 4.4: V3 Root Extraction ---
Verifying the new logic on 'butter', 'beef', and 'hummus':


,description,semantic_root
1139,"Beef, ground, 70% lean meat / 30% fat, patty, ...",beef ground
1868,"Pickles, cucumber, sweet, low sodium (includes...",pickles cucumber
409,"Pillsbury Grands, Buttermilk Biscuits, refrige...",pillsbury grands buttermilk biscuits
4948,"Peanut butter, chunky, vitamin and mineral for...",peanut butter chunky
4649,"Beef, ground, unspecified fat content, cooked",beef ground
587,"Creamy dressing, made with sour cream and/or b...",creamy dressing made
1961,"Beef, ground, 70% lean meat / 30% fat, crumble...",beef ground
177,"Candies, MARS SNACKFOOD US, M&M's Peanut Butte...",candies mars snackfood us
1784,"Squash, winter, butternut, cooked, baked, with...",squash winter
5157,"Biscuits, plain or buttermilk, refrigerated do...",biscuits plain or buttermilk


--------------------------------------------------------------------------------


In [15]:

# --- Verification (Semantic Root) ---
print("\n--- Verification: Semantic Root Extraction ---")
print("Verifying 'Nuts, almond butter...' example:")
display(usda_lookup_raw_df[usda_lookup_raw_df['description'].str.contains('Nuts, almond butter', case=False)][['description', 'semantic_root']].head())

print("\nVerifying 'Hummus, commercial' and 'Hummus - nfy...' example:")
display(usda_lookup_raw_df[usda_lookup_raw_df['description'].str.contains('Hummus', case=False)][['description', 'semantic_root']].head())
print("-" * 80)


--- Verification: Semantic Root Extraction ---
Verifying 'Nuts, almond butter...' example:


,description,semantic_root
1076,"Nuts, almond butter, plain, without salt added",almond butter
1091,"Nuts, almond butter, plain, with salt added",almond butter



Verifying 'Hummus, commercial' and 'Hummus - nfy...' example:


,description,semantic_root
4942,"Hummus, home prepared",hummus home prepared
6777,"Hummus, commercial",hummus commercial


--------------------------------------------------------------------------------


In [16]:
# --- 3. Thực hiện Groupby trên Gốc Ngữ nghĩa Mới ---
print("\n--- 3. Grouping by the new Semantic Root and Aggregating ---")
# Thay thế các giá trị 0 bằng NaN
usda_lookup_raw_df[numeric_cols] = usda_lookup_raw_df[numeric_cols].replace(0, np.nan)
# Groupby
usda_aggregated_lookup_df = usda_lookup_raw_df.groupby('semantic_root', as_index=False)[numeric_cols].mean()
# Điền 0
usda_aggregated_lookup_df.fillna(0, inplace=True)


--- 3. Grouping by the new Semantic Root and Aggregating ---


In [17]:
# --- 4. Lọc bỏ các thực phẩm không có Calo ---
print("\n--- 4. Filtering out foods with zero calories ---")
usda_aggregated_lookup_df = usda_aggregated_lookup_df[usda_aggregated_lookup_df['calories'] > 0]



--- 4. Filtering out foods with zero calories ---


In [18]:

# --- FINAL VALIDATION (V3) ---
print("\ninal Aggregated USDA Nutrient Lookup Dictionary (V3) created successfully.")
print("\n--- FINAL VALIDATION (V3) ---")
print(f"Shape of the final aggregated lookup table: {usda_aggregated_lookup_df.shape}")

print("\nVerifying 'almond butter' nutrient profile (should be a single, correct row):")
display(usda_aggregated_lookup_df[usda_aggregated_lookup_df['semantic_root'] == 'almond butter'])

print("\nVerifying 'hummus' nutrient profile (should be a single, correct row):")
display(usda_aggregated_lookup_df[usda_aggregated_lookup_df['semantic_root'] == 'hummus commercial'])


inal Aggregated USDA Nutrient Lookup Dictionary (V3) created successfully.

--- FINAL VALIDATION (V3) ---
Shape of the final aggregated lookup table: (2352, 7)

Verifying 'almond butter' nutrient profile (should be a single, correct row):


,semantic_root,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
21,almond butter,614.00,20.96,55.50,18.82,10.30,5.35



Verifying 'hummus' nutrient profile (should be a single, correct row):


,semantic_root,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
1039,hummus commercial,237.00,7.78,17.82,15.00,5.50,0.62


In [20]:
# --- FINAL VALIDATION ---
print("\nFinal Aggregated USDA Nutrient Lookup Dictionary created successfully.")
print("\n--- FINAL VALIDATION ---")
print(f"Shape of the final aggregated lookup table: {usda_aggregated_lookup_df.shape}")

print("\nVerifying 'hummus' nutrient profile (should be a single, complete row):")
display(usda_aggregated_lookup_df[usda_aggregated_lookup_df['semantic_root'].str.contains('hummus', na=False)])

print("\nVerifying 'butter' profile (should be a single, complete row):")
display(usda_aggregated_lookup_df[usda_aggregated_lookup_df['semantic_root'].str.contains('chicken ', na=False)])

print("\nFinal check for null values:")
if usda_aggregated_lookup_df.isnull().sum().sum() == 0:
    print("No null values found.")
else:
    print("WARNING: Null values still exist.")

print("\nDescriptive statistics of the final lookup table:")
display(usda_aggregated_lookup_df.describe())


Final Aggregated USDA Nutrient Lookup Dictionary created successfully.

--- FINAL VALIDATION ---
Shape of the final aggregated lookup table: (2352, 7)

Verifying 'hummus' nutrient profile (should be a single, complete row):


,semantic_root,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
1039,hummus commercial,237.00,7.78,17.82,15.00,5.50,0.62
1040,hummus home prepared,177.00,4.86,8.59,20.12,4.00,0.27



Verifying 'butter' profile (should be a single, complete row):


,semantic_root,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
35,applebee's chicken tenders,296.00,19.25,16.15,18.36,1.20,0.39
36,applebee's chicken tenders platter,297.00,19.62,16.24,17.98,1.00,0.36
269,burger king chicken strips,292.00,18.20,15.25,20.49,1.30,0.00
277,burger king original chicken sandwich,286.00,12.14,14.69,26.22,2.40,3.02
313,campbell's chunky classic chicken noodle soup,47.00,3.41,1.30,5.41,1.00,0.41
...,...,...,...,...,...,...,...
2006,soup healthy choice chicken and rice soup,37.00,2.53,0.55,5.71,0.80,0.37
2007,soup healthy choice chicken noodle soup,41.00,3.69,0.63,5.20,0.80,0.57
2110,subway oven roasted chicken sub on white bread,157.00,10.84,3.15,21.35,1.20,2.99
2114,subway sweet onion chicken teriyaki sub on whi...,155.00,10.91,2.37,22.54,1.20,6.06



Final check for null values:
No null values found.

Descriptive statistics of the final lookup table:


,calories,protein_g,fat_g,carbs_g,fiber_g,sugar_g
count,2352.00,2352.00,2352.00,2352.00,2352.00,2352.00
mean,236.55,8.42,11.24,26.56,2.61,10.30
std,182.79,9.59,18.21,26.28,5.01,16.07
min,1.00,0.00,0.00,0.00,0.00,0.00
25%,81.00,1.48,0.57,4.93,0.00,0.62
50%,214.00,5.22,4.42,16.56,1.30,3.41
75%,347.25,12.20,14.13,45.78,2.90,11.67
max,902.00,88.32,100.00,99.98,79.00,99.80


In [21]:
import os

# Định nghĩa đường dẫn thư mục dự án
project_dir = os.getcwd()  # Lấy thư mục làm việc hiện tại (thư mục dự án)
output_dir = os.path.join(project_dir, 'data_validation')  # Tạo thư mục con 'output' trong thư mục dự án

# Tạo thư mục 'output' nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lưu bảng dữ liệu vào file CSV
csv_path = os.path.join(output_dir, 'usda_aggregated_lookup.csv')
usda_aggregated_lookup_df.to_csv(csv_path, index=False)
print(f"Saved aggregated data to CSV: {csv_path}")

Saved aggregated data to CSV: /Users/nguyenthuong/Documents/DietPlanning/data_validation/usda_aggregated_lookup.csv


In [26]:
# Tế bào 5 (Phiên bản V2 - Tải và Làm sạch Toàn bộ Dữ liệu)
# --- Implementation & Verification ---
print("--- Step 5.1: Loading the entire RAW_recipes.csv dataset ---")
try:
    # Tải toàn bộ dữ liệu
    raw_recipes_df = pd.read_csv(os.path.join(FOOD_DOT_COM_PATH, 'RAW_recipes.csv'))
    print(f"Successfully loaded. Initial shape: {raw_recipes_df.shape}")
except Exception as e:
    print(f"ERROR loading RAW_recipes.csv: {e}")

--- Step 5.1: Loading the entire RAW_recipes.csv dataset ---
Successfully loaded. Initial shape: (231637, 12)


In [27]:

# --- Step 5.2: Initial Data Cleaning ---
print("\n--- Step 5.2: Performing initial data cleaning ---")
# Verification (Before): Kiểm tra giá trị trống ban đầu
print("\nNull values before cleaning:")
print(raw_recipes_df[['name', 'description', 'steps', 'ingredients']].isnull().sum())



--- Step 5.2: Performing initial data cleaning ---

Null values before cleaning:
name              1
description    4979
steps             0
ingredients       0
dtype: int64


In [28]:
# Loại bỏ các hàng có giá trị trống ở các cột thiết yếu
required_cols_for_processing = ['name', 'id', 'steps', 'ingredients', 'description']
recipes_cleaned_df = raw_recipes_df.dropna(subset=required_cols_for_processing).copy()
print(f"\nShape after dropping rows with null values: {recipes_cleaned_df.shape}")


Shape after dropping rows with null values: (226657, 12)


In [29]:
# Verification (After): Kiểm tra lại giá trị trống
print("\nNull values after cleaning:")
print(recipes_cleaned_df[required_cols_for_processing].isnull().sum())



Null values after cleaning:
name           0
id             0
steps          0
ingredients    0
description    0
dtype: int64


In [30]:

recipes_cleaned_df = recipes_cleaned_df.dropna(subset=required_cols_for_processing).copy()
recipes_cleaned_df.drop_duplicates(subset=['id'], inplace=True)
print(f"Initial cleaned shape for processing: {recipes_cleaned_df.shape}")
print("-" * 80)

Initial cleaned shape for processing: (226657, 12)
--------------------------------------------------------------------------------


In [31]:
# Kiểm tra và loại bỏ các hàng trùng lặp dựa trên 'id' của công thức
print(f"\nChecking for duplicate recipe IDs...")
initial_rows = len(recipes_cleaned_df)
recipes_cleaned_df.drop_duplicates(subset=['name'], inplace=True)
final_rows = len(recipes_cleaned_df)
print(f"Removed {initial_rows - final_rows} duplicate rows.")


Checking for duplicate recipe IDs...
Removed 1366 duplicate rows.


In [32]:

# --- FINAL VALIDATION (V3) ---
print("\ninal Aggregated USDA Nutrient Lookup Dictionary (V3) created successfully.")
print("\n--- FINAL VALIDATION (V3) ---")
print(f"Shape of the final aggregated lookup table: {recipes_cleaned_df.shape}")

print("\nVerifying 'almond butter' nutrient profile (should be a single, correct row):")
display(recipes_cleaned_df[recipes_cleaned_df['name'] == 'almond butter'])

print("\nVerifying 'hummus' nutrient profile (should be a single, correct row):")
display(recipes_cleaned_df[recipes_cleaned_df['name'] == 'hummus'])


inal Aggregated USDA Nutrient Lookup Dictionary (V3) created successfully.

--- FINAL VALIDATION (V3) ---
Shape of the final aggregated lookup table: (225291, 12)

Verifying 'almond butter' nutrient profile (should be a single, correct row):


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
3588,almond butter,369241,12,599450,2009-05-04,"['15-minutes-or-less', 'time-to-make', 'prepar...","[1094.8, 147.0, 36.0, 34.0, 77.0, 37.0, 13.0]",4,"['fit a food processor with the ""s"" blade and ...",based on a recipe from jennifer cornbleet's bo...,"['almonds', 'salt']",2



Verifying 'hummus' nutrient profile (should be a single, correct row):


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
109804,hummus,187591,105,190012,2006-09-26,"['time-to-make', 'course', 'main-ingredient', ...","[107.1, 5.0, 11.0, 10.0, 10.0, 3.0, 4.0]",5,"['soak chickpeas overnight', 'cook chickpeas f...",modified from alison & simon holsts vegetarian...,"['dried garbanzo beans', 'lemons, juice of', '...",7


In [33]:
# --- Final Verification for this step ---
print("\n--- Verification 5 ---")
print(f"Initial cleaning complete. Final shape for processing: {recipes_cleaned_df.shape}")
if recipes_cleaned_df[required_cols_for_processing].isnull().sum().sum() == 0:
    print("Confirmed: No null values in essential columns.")
if not recipes_cleaned_df['id'].duplicated().any():
    print("Confirmed: No duplicate recipe IDs.")


--- Verification 5 ---
Initial cleaning complete. Final shape for processing: (225291, 12)
Confirmed: No null values in essential columns.
Confirmed: No duplicate recipe IDs.


In [62]:
# TẾ BÀO 6 (PHIÊN BẢN V5 - HOÀN CHỈNH)

# --- 6.1: Hàm Chuẩn hóa (Giữ nguyên) ---
def normalize_ingredient(ing_name: str) -> str:
    """Chuẩn hóa chuỗi tên thành phần để tối ưu hóa việc tìm kiếm."""
    if not isinstance(ing_name, str):
        return ""
    ing_name = ing_name.lower()
    ing_name = ing_name.split(',')[0]
    ing_name = re.sub(r'(\d+/\d+|\d+\s\d+/\d+|\d+.\d+|\d+)\s*(oz|ounce|s|g|gram|s|kg|lb|s|cup|s|teaspoon|s|tablespoon|s|can|s|package|s|cloves|clove)\b', '', ing_name)
    ing_name = re.sub(r'\b(diced|chopped|fresh|minced|sliced|cooked|frozen|canned|peeled|seeded|to taste|optional|large|small|medium|thinly|roughly|pitted|crushed|undrained|drained|all-purpose)\b', '', ing_name)
    ing_name = re.sub(r'[^a-z\s-]', '', ing_name)
    words = ing_name.split()
    normalized_words = []
    for word in words:
        if word.endswith('es'):
            normalized_words.append(word[:-2])
        elif word.endswith('s'):
            normalized_words.append(word[:-1])
        else:
            normalized_words.append(word)
    return ' '.join(normalized_words).strip()

In [66]:
# --- 6.2: Hàm Xử lý Công thức (HOÀN CHỈNH V5) ---
def process_recipe_row_v5(row: pd.Series, lookup_df: pd.DataFrame) -> dict or None:
    """
    Xử lý một công thức, tính toán HỒ SƠ DINH DƯỠNG TRUNG BÌNH của các thành phần.
    """
    try:
        ingredients_list = eval(row['ingredients'])
        steps_list = eval(row['steps'])
    except Exception:
        return None

    matched_nutrients_list = []
    nutrient_cols = ['calories', 'protein_g', 'fat_g', 'carbs_g', 'fiber_g', 'sugar_g']

    for ing in ingredients_list:
        clean_ing = normalize_ingredient(ing)
        if not clean_ing or len(clean_ing) < 3:
            continue
        
        match = None
        # Ưu tiên 1: Tìm kiếm trùng khớp chính xác
        exact_match = lookup_df[lookup_df['semantic_root'] == clean_ing]
        if not exact_match.empty:
            match = exact_match.iloc[0]
        else:
            # Ưu tiên 2 (Cải tiến): Tìm kiếm sự chứa đựng (containment) và chọn kết quả tốt nhất
            # Tìm tất cả các 'semantic_root' có chứa 'clean_ing'
            # Ví dụ: 'chicken' trong công thức sẽ khớp với 'chicken breast' trong lookup
            # Ngược lại: 'white rice' trong công thức sẽ khớp với 'rice' trong lookup
            
            # Xây dựng một regex từ các từ trong thành phần
            words_in_ing = [re.escape(word) for word in clean_ing.split() if len(word) > 2]
            if not words_in_ing:
                continue
            
            # Regex tìm kiếm bất kỳ từ nào: \b(word1|word2|...)\b
            search_regex = r'\b(' + '|'.join(words_in_ing) + r')\b'
            
            partial_matches = lookup_df[lookup_df['semantic_root'].str.contains(search_regex, na=False, case=False)]
            
            if not partial_matches.empty:
                # Ưu tiên 3: Trong số các kết quả khớp, chọn cái ngắn nhất (cô đọng nhất)
                # Ví dụ: 'rice' sẽ được ưu tiên hơn 'rice flour'
                best_match_idx = partial_matches['semantic_root'].str.len().idxmin()
                match = partial_matches.loc[best_match_idx]
        if match is not None:
            matched_nutrients_list.append(match[nutrient_cols])
    
    # Điều kiện lọc: phải có ít nhất 2 thành phần hợp lệ
    if len(matched_nutrients_list) >= 2:
        nutrients_df = pd.DataFrame(matched_nutrients_list)
        
        # TÍNH TOÁN HỒ SƠ TRUNG BÌNH
        avg_profile = nutrients_df.mean()
        
        return {
            'name': row['name'],
            'description': row['description'],
            'steps': steps_list,
            'ingredients_list': ingredients_list,
            # Các đặc trưng này sẽ được dùng để huấn luyện mô hình
            'avg_calories': avg_profile['calories'],
            'avg_protein_g': avg_profile['protein_g'],
            'avg_fat_g': avg_profile['fat_g'],
            'avg_carbs_g': avg_profile['carbs_g'],
            'avg_fiber_g': avg_profile['fiber_g'],
            'avg_sugar_g': avg_profile['sugar_g']
        }
    return None

In [67]:

# --- 6.3: Unit Test cho phiên bản V5 ---
print("--- 6.3: Unit Testing the V5 Function ---")

test_row = pd.Series({
    'ingredients': "['1 lb chicken breast', '2 cups white rice', 'unknown item']",
    'steps': "['cook chicken and rice']", 'name': 'Test Meal', 'description': 'A simple test meal'
})

test_lookup = pd.DataFrame({
    'semantic_root': ['chicken breast', 'rice', 'broccoli'],
    'calories': [165.0, 130.0, 34.0], 'protein_g': [31.0, 2.7, 2.8], 'fat_g': [3.6, 0.3, 0.4],
    'carbs_g': [0.0, 28.0, 6.6], 'fiber_g': [0.0, 0.4, 2.6], 'sugar_g': [0.0, 0.1, 1.7]
})

print("Testing with a sample row using V5 logic...")
result_v5 = process_recipe_row_v5(test_row, test_lookup)

# Tính toán kết quả mong đợi (cả chicken breast và rice đều phải được tìm thấy)
expected_total_calories = 165.0 + 130.0 # = 295.0

if result_v5 and abs(result_v5['total_calories'] - expected_total_calories) < 0.01:
    print("Unit test PASSED!")
    print(f"   Total Calories Calculated: ~{result_v5['total_calories']:.2f} (Correct)")
    print(f"   Total Protein Calculated: ~{result_v5['total_protein_g']:.2f}")
    display(pd.Series(result_v5))
else:
    print("Unit test FAILED! Please review the V5 logic.")
    if result_v5:
        print(f"   Got Total Calories: {result_v5.get('total_calories')}, Expected: {expected_total_calories}")
    else:
        print("   Reason: The function returned None. Check matching logic and `len >= 1` condition.")

--- 6.3: Unit Testing the V5 Function ---
Testing with a sample row using V5 logic...


KeyError: 'total_calories'

In [68]:
total_calories# --- 6.3: Unit Test (Cực kỳ quan trọng) ---
print("--- 6.3: Unit Testing the V5 Function ---")
test_row = pd.Series({
    'ingredients': "['1 lb chicken breast', '2 cups white rice', '1/2 cup olive oil']",
    'steps': "['...']", 'name': 'Test Meal', 'description': '...'
})
test_lookup = pd.DataFrame({
    'semantic_root': ['chicken breast', 'rice', 'olive oil'],
    'calories':      [165.0,  130.0,  884.0],
    'protein_g':     [31.0,   2.7,    0.0],
    'fat_g':         [3.6,    0.3,    100.0],
    'carbs_g':       [0.0,    28.0,   0.0],
    'fiber_g':       [0.0,    0.4,    0.0],
    'sugar_g':       [0.0,    0.1,    0.0]
})

result = process_recipe_row_v5(test_row, test_lookup)

# Tính toán kết quả mong đợi
expected_avg_calories = (165.0 + 130.0 + 884.0) / 3
expected_avg_protein_g = (31.0 + 2.7 + 0.0) / 3

if result and abs(result['avg_calories'] - expected_avg_calories) < 0.01 and abs(result['avg_protein_g'] - expected_avg_protein_g) < 0.01:
    print(f"Unit test passed!")
    print(f"   Calculated Avg Calories: ~{result['avg_calories']:.2f} (Expected: ~{expected_avg_calories:.2f})")
    print(f"   Calculated Avg Protein: ~{result['avg_protein_g']:.2f} (Expected: ~{expected_avg_protein_g:.2f})")
    display(pd.Series(result))
else:
    print(" Unit test failed!")
    if result:
        print(f"   Got Avg Calories: {result.get('avg_calories')}, Expected: {expected_avg_calories}")
    else:
        print("   Reason: The function returned None. Condition 'len(matched_nutrients_list) >= 2' might not be met.")


NameError: name 'total_calories' is not defined

In [48]:
# TẾ BÀO 7: KIỂM TRA CHUẨN BỊ

print("--- Step 7: Preparing for Batch Processing ---")

# 1. Xác nhận sự tồn tại và hình dạng của các DataFrame cần thiết
print("\nVerifying required DataFrames...")
try:
    print(f"Shape of USDA Lookup Table: {usda_aggregated_lookup_df.shape}")
    print(f"Shape of Cleaned Recipes Table: {recipes_cleaned_df.shape}")
    if usda_aggregated_lookup_df.shape[0] > 0 and recipes_cleaned_df.shape[0] > 0:
        print("DataFrames are ready.")
    else:
        print("WARNING: One or more DataFrames are empty. Please re-run previous steps.")
except NameError as e:
    print(f"ERROR: A required DataFrame is missing: {e}. Please ensure all previous cells were run successfully.")
    
# 2. Xác nhận sự tồn tại của hàm xử lý
print("\nVerifying processing function...")
try:
    if callable(process_recipe_row_v5):
        print("Function 'process_recipe_row_v5' is ready.")
    else:
        print("WARNING: 'process_recipe_row_v5' is not a function.")
except NameError:
    print("process_recipe_row_v5ERROR: Function 'process_recipe_row_v5' not found. Please define it first.")

print("\n--- Preparation complete. Ready for batch execution. ---")

--- Step 7: Preparing for Batch Processing ---

Verifying required DataFrames...
Shape of USDA Lookup Table: (2352, 7)
Shape of Cleaned Recipes Table: (225291, 12)
DataFrames are ready.

Verifying processing function...
Function 'process_recipe_row_v5' is ready.

--- Preparation complete. Ready for batch execution. ---


In [49]:
# TẾ BÀO 6.5: XỬ LÝ TOÀN BỘ DỮ LIỆU THEO LÔ (BATCH PROCESSING)

print("--- Step 6.5: Processing the Full Dataset in Batches ---")
c

--- Step 6.5: Processing the Full Dataset in Batches ---


In [51]:

CHUNK_SIZE = 10000  # Xử lý 10,000 công thức mỗi lần
all_processed_data = []

In [52]:
print(f"--- 3.3: Applying Final Logic to Full Dataset in Chunks ---")
print(f"Starting to process {len(recipes_cleaned_df)} recipes in chunks of {CHUNK_SIZE}...")
print("This will take a significant amount of time, please be patient.")

# Tạo một vòng lặp qua các chunk của DataFrame
for start in tqdm(range(0, len(recipes_cleaned_df), CHUNK_SIZE), desc="Overall Progress"):
    end = start + CHUNK_SIZE
    # Lấy ra một chunk để xử lý
    chunk_df = recipes_cleaned_df.iloc[start:end]
    
    # Áp dụng hàm xử lý trên chunk hiện tại. Thanh tiến trình sẽ hiển thị cho từng chunk.
    processed_chunk = chunk_df.progress_apply(
        process_recipe_row_final, 
        axis=1, 
        lookup_df=usda_aggregated_lookup_df
    )
    
    # Thêm các kết quả hợp lệ (không phải None) vào danh sách tổng
    all_processed_data.extend(processed_chunk.dropna().tolist())
    
    # Giải phóng bộ nhớ (tùy chọn nhưng là thói quen tốt khi xử lý dữ liệu lớn)
    import gc
    del chunk_df, processed_chunk
    gc.collect()

print("\nCreating the raw DataFrame from all processed chunks...")
clean_recipes_df_raw = pd.DataFrame(all_processed_data)

--- 3.3: Applying Final Logic to Full Dataset in Chunks ---
Starting to process 225291 recipes in chunks of 10000...
This will take a significant amount of time, please be patient.


Overall Progress:   0%|          | 0/23 [00:00<?, ?it/s]

Processing Chunk:   0%|          | 0/10000 [00:00<?, ?it/s]

Processing Chunk:   0%|          | 0/10000 [00:00<?, ?it/s]

Processing Chunk:   0%|          | 0/10000 [00:00<?, ?it/s]

...

Processing Chunk:   0%|          | 0/10000 [00:00<?, ?it/s]

Processing Chunk:   0%|          | 0/10000 [00:00<?, ?it/s]

Processing Chunk:   0%|          | 0/10000 [00:00<?, ?it/s]

Processing Chunk:   0%|          | 0/5291 [00:00<?, ?it/s]


Creating the raw DataFrame from all processed chunks...


In [53]:
# --- Verification 3.3 ---
print(f"\n--- Verification 3.3 ---")
print(f"Raw processing complete.")
if not clean_recipes_df_raw.empty:
    print(f"Total valid recipes created after processing: {len(clean_recipes_df_raw)}")
    print("First 5 rows of the raw processed data:")
    display(clean_recipes_df_raw.head())
    print("\nChecking for null values in raw processed data:")
    print(clean_recipes_df_raw.isnull().sum())
else:
    print("CRITICAL ERROR: No valid recipes were found. The processing function might have failed on all inputs.")
print("-" * 80)


--- Verification 3.3 ---
Raw processing complete.
Total valid recipes created after processing: 222733
First 5 rows of the raw processed data:


,name,description,steps,ingredients_list,total_calories,total_protein_g,total_fat_g,total_carbs_g,total_fiber_g,total_sugar_g
0,arriba baked winter squash mexican style,autumn is my favorite time of year to cook! th...,"[make a choice and proceed with recipe, depend...","[winter squash, mexican seasoning, mixed spice...",2983.50,50.96,242.19,175.22,22.59,102.92
1,a bit different breakfast pizza,this recipe calls for the crust to be prebaked...,"[preheat oven to 425 degrees f, press dough in...","[prepared pizza crust, sausage patty, eggs, mi...",1362.33,90.35,83.68,62.85,7.40,51.83
2,all in the kitchen chili,this modified version of 'mom's' chili was a h...,"[brown ground beef in large pot, add chopped o...","[ground beef, yellow onions, diced tomatoes, t...",1785.30,132.48,55.18,220.80,30.78,120.86
3,alouette potatoes,"this is a super easy, great tasting, make ahea...",[place potatoes in a large pot of lightly salt...,"[spreadable cheese with garlic and herbs, new ...",1751.33,43.88,118.70,133.03,17.90,59.47
4,amish tomato ketchup for canning,my dh's amish mother raised him on this recipe...,"[mix all ingredients& boil for 2 1 / 2 hours ,...","[tomato juice, apple cider vinegar, sugar, sal...",948.67,37.62,27.94,134.46,12.25,61.90



Checking for null values in raw processed data:
name                0
description         0
steps               0
ingredients_list    0
total_calories      0
total_protein_g     0
total_fat_g         0
total_carbs_g       0
total_fiber_g       0
total_sugar_g       0
dtype: int64
--------------------------------------------------------------------------------


In [54]:

# --- Implementation ---
print(f"--- 3.4: Final Feature Engineering and Cleaning ---")
# Tạo một bản sao để làm việc, giữ lại DataFrame thô ban đầu
df = clean_recipes_df_raw.copy()

--- 3.4: Final Feature Engineering and Cleaning ---


In [50]:
# Khởi tạo tqdm cho pandas
tqdm.pandas(desc="Processing Chunk")

In [55]:

# --- 3.4.1: Tạo các Đặc trưng Tỷ lệ & Mật độ ---
print("\n--- 3.4.1: Creating Ratio and Density Features for ML ---")
# Thêm một hằng số nhỏ (epsilon) để tránh lỗi chia cho 0 trong trường hợp total_calories = 0
epsilon = 1e-6
df['protein_energy_ratio'] = (df['total_protein_g'] * 4) / (df['total_calories'] + epsilon) * 100
df['fat_energy_ratio'] = (df['total_fat_g'] * 9) / (df['total_calories'] + epsilon) * 100
df['carbs_energy_ratio'] = (df['total_carbs_g'] * 4) / (df['total_calories'] + epsilon) * 100
df['sugar_density'] = (df['total_sugar_g'] / (df['total_calories'] + epsilon)) * 1000 # g per 1000 kcal


--- 3.4.1: Creating Ratio and Density Features for ML ---


In [57]:

# --- 3.4.2: Lọc các Dòng không hợp lệ (Sanity Check) ---
print("\n--- 3.4.2: Filtering Invalid Rows ---")
df['total_ratio'] = df['protein_energy_ratio'] + df['fat_energy_ratio'] + df['carbs_energy_ratio']
initial_rows = len(df)
# Giữ lại các công thức có tổng tỷ lệ năng lượng hợp lý (từ 80% đến 120%)
# và có total_calories > 50 (đây là một bước kiểm tra an toàn kép)
df_filtered = df[(df['total_ratio'] >= 80) & (df['total_ratio'] <= 120) & (df['total_calories'] > 50)].copy()

# --- Verification (Filtering) ---
print(f"Removed {initial_rows - len(df_filtered)} rows with imbalanced energy ratios or low calories.")
print(f"Shape after filtering: {df_filtered.shape}")
print("-" * 60)


--- 3.4.2: Filtering Invalid Rows ---
Removed 2377 rows with imbalanced energy ratios or low calories.
Shape after filtering: (220356, 15)
------------------------------------------------------------


In [56]:
# --- Verification (Ratio Features) ---
print("New features created. Verifying statistics:")
ratio_features = ['protein_energy_ratio', 'fat_energy_ratio', 'carbs_energy_ratio', 'sugar_density']
display(df[ratio_features].describe())
print("-" * 60)

New features created. Verifying statistics:


,protein_energy_ratio,fat_energy_ratio,carbs_energy_ratio,sugar_density
count,222733.00,222733.00,222733.00,222733.00
mean,13.41,49.39,40.57,44.88
std,9.03,21.35,20.06,36.56
min,0.00,0.00,0.00,0.00
25%,7.12,34.38,25.91,17.33
50%,11.42,51.89,38.57,35.87
75%,17.56,65.14,52.59,63.43
max,87.63,108.30,158.18,333.00


------------------------------------------------------------


In [58]:

# --- 3.4.3: Làm sạch Cột 'name' ---
print("\n--- 3.4.3: Cleaning the 'name' column ---")
def clean_recipe_name(name: str) -> str:
    """Làm sạch tên công thức để hiển thị."""
    if not isinstance(name, str): return "Untitled Recipe"
    # Loại bỏ các tiền tố không mang nhiều ý nghĩa
    name = re.sub(r'^(a bit different|all in the kitchen|absolute best|almost famous|another|awesome|best ever|crazy good|my favorite|super easy|the ultimate)\s+', '', name, flags=re.IGNORECASE)
    # Viết hoa chữ cái đầu của câu
    return name.capitalize()

df_filtered['name'] = df_filtered['name'].apply(clean_recipe_name)


--- 3.4.3: Cleaning the 'name' column ---


In [59]:

# --- Verification (Clean Name) ---
print("Cleaned 'name' column.")
print("Sample comparison of original vs. cleaned names:")
# Lấy index của 5 hàng đầu tiên trong df_filtered
sample_indices = df_filtered.head().index
# Tìm các tên gốc tương ứng trong clean_recipes_df_raw
original_names_sample = clean_recipes_df_raw.loc[sample_indices]['name']
cleaned_names_sample = df_filtered.head()['name']
comparison = pd.DataFrame({'original_name': original_names_sample, 'cleaned_name': cleaned_names_sample})
display(comparison)
print("-" * 60)


Cleaned 'name' column.
Sample comparison of original vs. cleaned names:


,original_name,cleaned_name
0,arriba baked winter squash mexican style,Arriba baked winter squash mexican style
1,a bit different breakfast pizza,Breakfast pizza
2,all in the kitchen chili,Chili
3,alouette potatoes,Alouette potatoes
4,amish tomato ketchup for canning,Amish tomato ketchup for canning


------------------------------------------------------------


In [60]:


# --- 3.4.4: Lựa chọn Cột Cuối cùng ---
print("\n--- 3.4.4: Selecting Final Columns for the Dataset ---")
# Các đặc trưng này sẽ được dùng để huấn luyện K-Means
training_features = ['protein_energy_ratio', 'fat_energy_ratio', 'carbs_energy_ratio', 'sugar_density']

# Các cột khác để lưu vào DB cho ứng dụng sử dụng
display_and_app_cols = [
    'name', 'description', 'steps', 'ingredients_list',
    'total_calories', 'total_protein_g', 'total_fat_g', 'total_carbs_g', 'total_fiber_g', 'total_sugar_g'
]

final_df = df_filtered[display_and_app_cols + training_features]


--- 3.4.4: Selecting Final Columns for the Dataset ---


In [61]:

# --- FINAL VALIDATION ---
print("\nFinal Clean and Engineered DataFrame (V4) created successfully.")
print("\n--- FINAL VALIDATION ---")
print(f"Shape of the final DataFrame: {final_df.shape}")
print("\nFinal Columns:", final_df.columns.tolist())
print("\nFinal Descriptive Statistics (for ML Training Features):")
display(final_df[training_features].describe())
print("\nFirst 5 rows of the final DataFrame:")
display(final_df.head())


Final Clean and Engineered DataFrame (V4) created successfully.

--- FINAL VALIDATION ---
Shape of the final DataFrame: (220356, 14)

Final Columns: ['name', 'description', 'steps', 'ingredients_list', 'total_calories', 'total_protein_g', 'total_fat_g', 'total_carbs_g', 'total_fiber_g', 'total_sugar_g', 'protein_energy_ratio', 'fat_energy_ratio', 'carbs_energy_ratio', 'sugar_density']

Final Descriptive Statistics (for ML Training Features):


,protein_energy_ratio,fat_energy_ratio,carbs_energy_ratio,sugar_density
count,220356.00,220356.00,220356.00,220356.00
mean,13.40,49.72,40.31,44.40
std,9.01,21.19,19.81,36.08
min,0.00,0.00,0.00,0.00
25%,7.13,35.09,25.79,17.17
50%,11.41,52.15,38.38,35.52
75%,17.53,65.30,52.32,62.78
max,87.63,108.30,115.68,303.86



First 5 rows of the final DataFrame:


,name,description,steps,ingredients_list,total_calories,total_protein_g,total_fat_g,total_carbs_g,total_fiber_g,total_sugar_g,protein_energy_ratio,fat_energy_ratio,carbs_energy_ratio,sugar_density
0,Arriba baked winter squash mexican style,autumn is my favorite time of year to cook! th...,"[make a choice and proceed with recipe, depend...","[winter squash, mexican seasoning, mixed spice...",2983.50,50.96,242.19,175.22,22.59,102.92,6.83,73.06,23.49,34.50
1,Breakfast pizza,this recipe calls for the crust to be prebaked...,"[preheat oven to 425 degrees f, press dough in...","[prepared pizza crust, sausage patty, eggs, mi...",1362.33,90.35,83.68,62.85,7.40,51.83,26.53,55.28,18.45,38.05
2,Chili,this modified version of 'mom's' chili was a h...,"[brown ground beef in large pot, add chopped o...","[ground beef, yellow onions, diced tomatoes, t...",1785.30,132.48,55.18,220.80,30.78,120.86,29.68,27.82,49.47,67.70
3,Alouette potatoes,"this is a super easy, great tasting, make ahea...",[place potatoes in a large pot of lightly salt...,"[spreadable cheese with garlic and herbs, new ...",1751.33,43.88,118.70,133.03,17.90,59.47,10.02,61.00,30.38,33.96
4,Amish tomato ketchup for canning,my dh's amish mother raised him on this recipe...,"[mix all ingredients& boil for 2 1 / 2 hours ,...","[tomato juice, apple cider vinegar, sugar, sal...",948.67,37.62,27.94,134.46,12.25,61.90,15.86,26.50,56.69,65.25


In [ ]:
# TẾ BÀO 6.5: XỬ LÝ TOÀN BỘ DỮ LIỆU THEO LÔ (BATCH PROCESSING)

# --- Objective ---
# Áp dụng hàm `process_recipe_row_v5` lên toàn bộ file RAW_recipes.csv
# mà không làm quá tải bộ nhớ RAM 8GB bằng cách đọc và xử lý theo từng lô.
# Kết quả của mỗi lô sẽ được lưu tạm thời, sau đó được hợp nhất lại.



# --- 1. Cấu hình và Chuẩn bị ---
CHUNK_SIZE = 20000  # Xử lý 20,000 công thức mỗi lần. Em có thể điều chỉnh con số này.
RAW_RECIPES_PATH = os.path.join(FOOD_DOT_COM_PATH, 'RAW_recipes.csv')
OUTPUT_DIR = 'data_validation' # Thư mục để lưu các file kết quả tạm thời

# Tạo thư mục output nếu chưa tồn tại
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created directory: {OUTPUT_DIR}")


In [ ]:

# --- 2. Xử lý lặp qua các lô ---
# Tạo một iterator để đọc file CSV theo từng chunk
csv_iterator = pd.read_csv(RAW_RECIPES_PATH, chunksize=CHUNK_SIZE)

# Lấy tổng số dòng để tqdm hiển thị thanh tiến trình chính xác
total_rows = len(recipes_cleaned_df) # Sử dụng df đã làm sạch từ Tế bào 5
num_chunks = (total_rows // CHUNK_SIZE) + 1

print(f"\nStarting to process {total_rows} recipes in {num_chunks} chunks...")

for i, chunk_df in enumerate(tqdm(csv_iterator, total=num_chunks, desc="Processing Chunks")):
    
    # Áp dụng hàm xử lý cho lô hiện tại
    # Lưu ý: progress_apply không cần thiết ở đây vì tqdm đã bao ở ngoài
    processed_data = chunk_df.apply(
        lambda row: process_recipe_row_v5(row, usda_aggregated_lookup_df), 
        axis=1
    )
    
    # Lọc bỏ các kết quả None
    valid_results = [item for item in processed_data if item is not None]
    
    # Nếu lô này có kết quả hợp lệ, chuyển thành DataFrame và lưu lại
    if valid_results:
        processed_chunk_df = pd.DataFrame(valid_results)
        
        # Lưu dưới dạng Parquet - hiệu quả hơn CSV về tốc độ và dung lượng
        output_path = os.path.join(OUTPUT_DIR, f'chunk_{i}.parquet')
        processed_chunk_df.to_parquet(output_path, index=False)

print("\nBatch processing complete. All chunks have been processed and saved.")


# --- 3. Hợp nhất tất cả các lô đã xử lý ---
print("\n--- Merging all processed chunks into a final DataFrame ---")

processed_files = [os.path.join(OUTPUT_DIR, f) for f in os.listdir(OUTPUT_DIR) if f.endswith('.parquet')]
df_list = [pd.read_parquet(file) for file in processed_files]

# Hợp nhất tất cả các DataFrame trong list lại
recipes_df = pd.concat(df_list, ignore_index=True)

# --- Verification ---
print("\n--- Final Verification ---")
print("Successfully merged all chunks.")
print(f"Shape of the final, fully processed DataFrame: {recipes_df.shape}")
print("Info of the final DataFrame:")
recipes_df.info()

print("\nDisplaying a sample of the final data:")
display(recipes_df.head())

# Dọn dẹp các file chunk tạm thời (Tùy chọn)
# import shutil
# shutil.rmtree(OUTPUT_DIR)
# print(f"\nRemoved temporary directory: {OUTPUT_DIR}")

In [ ]:
# TẾ BÀO 7: KỸ THUẬT ĐẶC TRƯNG (FEATURE ENGINEERING) - PHIÊN BẢN CẬP NHẬT

# --- Objective ---
# Sử dụng DataFrame `recipes_df` đã được xử lý đầy đủ từ Bước 6.5
# để thực hiện Feature Engineering.

print("--- Step 7: Feature Engineering on the Full Dataset ---")

# Giờ đây, chúng ta không cần tạo mẫu nữa.
# DataFrame `recipes_df` đã tồn tại từ tế bào trước và chứa toàn bộ dữ liệu đã xử lý.
if 'recipes_df' not in locals() or recipes_df.empty:
    print("ERROR: DataFrame 'recipes_df' not found or is empty.")
    print("Please make sure you have successfully run the Batch Processing step (6.5) first.")
else:
    print(f"Input DataFrame 'recipes_df' found with shape: {recipes_df.shape}")

    # --- 7.1: Tính toán Tỷ lệ Năng lượng Macros ---
    print("\n--- 7.1: Calculating Macronutrient Energy Ratios ---")

    # Tạo bản sao để tránh SettingWithCopyWarning
    recipes_features_df = recipes_df.copy()

    # Hằng số calo/gram
    CALS_PER_G_PROTEIN = 4
    CALS_PER_G_CARB = 4
    CALS_PER_G_FAT = 9

    # Tính tổng calo từ macros (để chuẩn hóa và kiểm tra tính hợp lệ)
    # Thêm 1e-6 để tránh lỗi chia cho 0
    recipes_features_df['macro_calories'] = (
        recipes_features_df['total_protein_g'] * CALS_PER_G_PROTEIN +
        recipes_features_df['total_carbs_g'] * CALS_PER_G_CARB +
        recipes_features_df['total_fat_g'] * CALS_PER_G_FAT + 1e-6
    )

    # Tính tỷ lệ % năng lượng từ mỗi macro
    recipes_features_df['protein_percent'] = (recipes_features_df['total_protein_g'] * CALS_PER_G_PROTEIN) / recipes_features_df['macro_calories'] * 100
    recipes_features_df['fat_percent'] = (recipes_features_df['total_fat_g'] * CALS_PER_G_FAT) / recipes_features_df['macro_calories'] * 100
    recipes_features_df['carbs_percent'] = (recipes_features_df['total_carbs_g'] * CALS_PER_G_CARB) / recipes_features_df['macro_calories'] * 100

    # --- Verification 7.1 ---
    print("\n--- Verification 7.1 ---")
    print("Displaying new percentage features for the first 5 recipes:")
    display(recipes_features_df[[
        'name', 'total_calories', 
        'protein_percent', 'fat_percent', 'carbs_percent'
    ]].head())

    # --- 7.2: Làm sạch dữ liệu sau khi tính toán ---
    print("\n--- 7.2: Cleaning Data Post-Calculation ---")

    initial_rows = len(recipes_features_df)
    recipes_features_df = recipes_features_df[recipes_features_df['total_calories'] > 50].copy()
    print(f"Removed {initial_rows - len(recipes_features_df)} rows with calories <= 50.")

    initial_rows = len(recipes_features_df)
    recipes_features_df = recipes_features_df[
        (recipes_features_df['protein_percent'] <= 100) &
        (recipes_features_df['fat_percent'] <= 100) &
        (recipes_features_df['carbs_percent'] <= 100)
    ].copy()
    print(f"Removed {initial_rows - len(recipes_features_df)} rows with unrealistic percentage values.")

    # --- Verification 7.2 ---
    print("\n--- Verification 7.2 ---")
    print(f"Final shape of the feature-engineered DataFrame: {recipes_features_df.shape}")
    print("Descriptive statistics of the cleaned data:")
    display(recipes_features_df[['total_calories', 'protein_percent', 'fat_percent', 'carbs_percent']].describe())

    # Dọn dẹp các cột không cần thiết
    recipes_features_df.drop(columns=['macro_calories'], inplace=True, errors='ignore')
    print("\nCleaned up intermediate columns.")